Diego Toribio <br>
Professor Fred Fontaine <br>
EID-378 Finance <br>
Problem Set III: MPT and Risk<br>

In [2]:
!pip install yfinance --quiet

In [3]:
import yfinance as yf
import pandas as pd
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

In this assignment, you will be exploring:

- Modern Portfolio Theory, including concepts such as the \( \beta \) and Sharpe ratio.
- V@R and CV@R risk measures, including their application to portfolio optimization (as an alternative to MPT).

## Section 1 - Preparing the Data

1. Select **M = 5** stocks, and get the daily adjusted closing price over one year for them.

2. Also get the S&P 500 index (daily) over the same year.

3. We also want to determine a risk-free return to use for our portfolio models, based on 3-Month Treasury Bills. This will be explained separately.

4. Compute the daily returns of the stocks and S&P 500 over the year. In order to have a return for each trading day, you should also grab the last trading day of the previous year. For example, for 2014, you should also grab Dec. 31, 2013, in order to compute the return for the first trading day (Jan. 2) of 2014. This idea is called 'pre-windowing.' It is fine if you determine the last trading day of the previous year "manually" (i.e., rather than writing systematic code that would "always" work).

5. Subtract off the daily risk-free return from the daily returns of the stocks and S&P 500. Going forward, we are going to be working with these excess returns only (I will omit using the word "excess" but keep in mind, everything is excess!). For example, if we think in terms of the Markowitz bullet in the $ (\sigma, \mu) $ plane, the 'risk-free' point in the graph will actually be the origin (0,0). More generally, going forward in the analysis, you can link what is happening here back to the "theory" discussed in the lecture notes by acting as if the risk-free return is 0!


#### 1.1 – Adjusted Close Data Retrieval  
We include the last trading day of the prior year so that our first return can be computed on the first trading day of the target year. Here we download auto‑adjusted close prices for our $M=5$ stocks.


In [5]:
# download adjusted close prices
start_date = "2021-12-31"
end_date = "2022-12-31"
tickers = ["AAPL", "MSFT", "GOOG", "AMZN", "TSLA"]
stock_data = yf.download(
    tickers, start=start_date, end=end_date, progress=False, auto_adjust=True
)["Close"]

# prepare first five rows for display
df_head = stock_data.head()
rows = [
    [date.strftime("%Y-%m-%d")] + [f"{v:.2f}" for v in row]
    for date, row in df_head.iterrows()
]

print(
    "\n\033[1mTable 1\033[0m. Adjusted Close Prices for Selected Stocks (first five rows)"
)
print(tabulate(rows, headers=["Date"] + list(df_head.columns), tablefmt="pretty"))


Table 1. Adjusted Close Prices for Selected Stocks (first five rows)
+------------+--------+--------+--------+--------+--------+
|    Date    |  AAPL  |  AMZN  |  GOOG  |  MSFT  |  TSLA  |
+------------+--------+--------+--------+--------+--------+
| 2021-12-31 | 174.52 | 166.72 | 144.00 | 327.16 | 352.26 |
| 2022-01-03 | 178.88 | 170.40 | 144.39 | 325.63 | 399.93 |
| 2022-01-04 | 176.61 | 167.52 | 143.74 | 320.05 | 383.20 |
| 2022-01-05 | 171.91 | 164.36 | 137.00 | 307.76 | 362.71 |
| 2022-01-06 | 169.04 | 163.25 | 136.90 | 305.33 | 354.90 |
+------------+--------+--------+--------+--------+--------+


#### 1.2 – S&P 500 Benchmark Retrieval  
Using the same date range (with pre‑window), we pull the S&P 500 adjusted close series and tabulate its first five observations.

In [7]:
# download S&P 500 “Adj Close” over the same date range
sp500 = yf.download(
    "^GSPC", start=start_date, end=end_date, progress=False, auto_adjust=False
)["Adj Close"]

# prepare first five rows, round prices to 2 decimals
df_sp500 = sp500.head().reset_index()
df_sp500.columns = ["Date", "Adj Close"]
rows = [
    [row["Date"].strftime("%Y-%m-%d"), f"{row['Adj Close']:.2f}"]
    for _, row in df_sp500.iterrows()
]

print("\n\033[1mTable 2\033[0m. S&P 500 Adj Close (first five rows, 2 dp).")
print(tabulate(rows, headers=["Date", "Adj Close"], tablefmt="pretty"))


Table 2. S&P 500 Adj Close (first five rows, 2 dp).
+------------+-----------+
|    Date    | Adj Close |
+------------+-----------+
| 2021-12-31 |  4766.18  |
| 2022-01-03 |  4796.56  |
| 2022-01-04 |  4793.54  |
| 2022-01-05 |  4700.58  |
| 2022-01-06 |  4696.05  |
+------------+-----------+


### 1.3 – Per‑Day Risk‑Free Rate

Assuming an annual T‑Bill yield of $2\%$, we convert it to a per‑trading‑day rate by  

$$
r_{\text{daily}} = (1 + r_{\text{ann}})^{1/252} - 1,
$$

and broadcast this constant across our date index to serve as the baseline for excess‑return computations.

In [8]:
# compute daily risk‑free rate from 2% annual T‑Bill
annual_rf_rate = 0.02
daily_rf_rate = (1 + annual_rf_rate) ** (1 / 252) - 1

# create series for subtraction
risk_free_daily = pd.Series(daily_rf_rate, index=stock_data.index)

print(f"Daily risk‑free rate (per trading day): {daily_rf_rate:.6e}")

Daily risk‑free rate (per trading day): 7.858494e-05


### 1.4 – Daily Return

We compute each series’ raw returns by  

$$
r_t = \frac{P_t - P_{t-1}}{P_{t-1}},
$$

then discard the initial missing value to produce a continuous daily return series for our analysis.

In [9]:
# compute daily returns
stock_returns = stock_data.pct_change().dropna()
sp500_returns = sp500.pct_change().dropna()

# first five rows of stock returns
print("\n\033[1mTable 3\033[0m. Stock daily returns (first five rows)")
rows = [
    [date.strftime("%Y-%m-%d")] + [f"{v:.4f}" for v in row]
    for date, row in stock_returns.head(5).iterrows()
]
print(tabulate(rows, headers=["Date"] + list(stock_returns.columns), tablefmt="pretty"))

# first five rows of S&P 500 returns
df_sp_ret = sp500_returns.head(5).reset_index()
df_sp_ret.columns = ["Date", "SP500 Return"]
rows = [
    [row["Date"].strftime("%Y-%m-%d"), f"{row['SP500 Return']:.4f}"]
    for _, row in df_sp_ret.iterrows()
]
print("\n\033[1mTable 4\033[0m. S&P 500 daily returns (first five rows)")
print(tabulate(rows, headers=["Date", "SP500 Return"], tablefmt="pretty"))


Table 3. Stock daily returns (first five rows)
+------------+---------+---------+---------+---------+---------+
|    Date    |  AAPL   |  AMZN   |  GOOG   |  MSFT   |  TSLA   |
+------------+---------+---------+---------+---------+---------+
| 2022-01-03 | 0.0250  | 0.0221  | 0.0027  | -0.0047 | 0.1353  |
| 2022-01-04 | -0.0127 | -0.0169 | -0.0045 | -0.0171 | -0.0418 |
| 2022-01-05 | -0.0266 | -0.0189 | -0.0468 | -0.0384 | -0.0535 |
| 2022-01-06 | -0.0167 | -0.0067 | -0.0007 | -0.0079 | -0.0215 |
| 2022-01-07 | 0.0010  | -0.0043 | -0.0040 | 0.0005  | -0.0354 |
+------------+---------+---------+---------+---------+---------+

Table 4. S&P 500 daily returns (first five rows)
+------------+--------------+
|    Date    | SP500 Return |
+------------+--------------+
| 2022-01-03 |    0.0064    |
| 2022-01-04 |   -0.0006    |
| 2022-01-05 |   -0.0194    |
| 2022-01-06 |   -0.0010    |
| 2022-01-07 |   -0.0041    |
+------------+--------------+


## Section 2 - Data Analysis
###  Modern Portfolio Theory

#### Some Equations

Let us write some equations (which apply in this form—with the risk-free return already subtracted off). Let $ C $ denote the covariance of the returns of the risky stocks (not the S&P 500), $ \mathbf{m} $ the vector of their expected (excess) returns. Given a portfolio weight vector $ \mathbf{w} $ (satisfying $\sum w_k = 1$), the expected (excess) return and standard deviation for $ \mathbf{w} $ is:

$$
\mu(\mathbf{w}) = \mathbf{m}^T\mathbf{w} \\
\sigma(\mathbf{w}) = \sqrt{\mathbf{w}^T C \mathbf{w}}
$$

Also let $ \mathbf{1} $ denote the vector of all ones, as usual.  
The weight vectors for the MVP and MP are:

$$
\mathbf{w}_{MVP} = \frac{1}{\mathbf{1}^T C^{-1}\mathbf{1}} C^{-1}\mathbf{1} \\
\mathbf{w}_{MP} = \frac{1}{\mathbf{1}^T C^{-1}\mathbf{m}} C^{-1}\mathbf{m}
$$

From these you can compute the $\mu, \sigma$.

If you have two portfolios (of the risky stocks only) with weight vectors $ \mathbf{w}_1, \mathbf{w}_2 $, the corresponding returns $ r_1, r_2 $ have covariance given by:

$$
cov(r_1, r_2) = \mathbf{w}_1^T C \mathbf{w}_2
$$

Otherwise, if you have the actual set of returns over a period of time, you can compute the covariance directly from the data. For example, you can do this to compute the covariance between one of your stocks and the S&P 500. If you store the covariances between each stock and the S&P 500 in a vector $ \mathbf{q} $, then the covariance between a portfolio $ \mathbf{w} $ of the stocks and the S&P 500 would be:

$$
cov(r_{\mathbf{w}}, r_{S\&P500}) = \mathbf{q}^T\mathbf{w}
$$



1. Using the data for the full year, we want to get the vector of expected returns and covariance matrix of the returns of the **M = 5** stocks, and the covariance between each stock and the S&P 500. Compute $ \mathbf{w}_{MVP}, (\sigma_{MVP}, \mu_{MVP}) $ and $ \mathbf{w}_{MP}, (\sigma_{MP}, \mu_{MP}) $ for the minimum variance and market portfolios, respectively. We will also need code that, given a weight vector for a portfolio from the stocks $ \mathbf{w} $, we can compute the $ (\sigma,\mu) $ point; also that we can compute the covariance between two such portfolios; we also want the vector $ \mathbf{q} $ of covariances with the S&P 500, and the ability to compute the covariance of a portfolio given by $ \mathbf{w} $ with the S&P 500.

2. Look at the MVP and MP weight vectors. Do they involve short-selling?

3. Compute the Sharpe ratio and $ \beta $ for each of the **M = 5** stocks, the S&P 500, and the MVP. Compute the covariance between the individual stocks, and the MVP and MP portfolios with the S&P 500. Display these results in a reasonable form, such as a table. Also provide a quick summary of which of these are positively or negatively correlated with the S&P 500.

4. Generate **N = 100** random $ \mathbf{w} $ vectors with **no short selling**; they have to satisfy the constraint that $\sum_{k=1}^{M} w_k = 1$. To do this I suggest: (1) generate $ M $ independent numbers uniformly from 0 to 1, say $(x_1, x_2, \dots, x_5)$, and then (2) set $ w_k = x_k / (\sum x_k) $, i.e., normalize them to sum to 1. I suggest you store these vectors in an array (dataframe) as you will be using them again later!

5. Compute the $ (\sigma, \mu) $ point for each of these vectors, and create a scatter plot of these $ (\sigma, \mu) $, also put in $ (\sigma_{MVP}, \mu_{MVP}) $, and $ (\sigma_{MP}, \mu_{MP}) $ and also compute and plot $ (\sigma_{SP}, \mu_{SP}) $ for the S&P 500. Also draw the CAL on your graph (remember, the "risk-free" point is now (0,0)).

6. Find the median of the expected returns of those 100 portfolios that involve no short-selling, say $ \mu_{med} $. Find the "best" portfolio (among those you created) whose return satisfies $ \mu \geq \mu_{med} $, i.e., among all such, the $ \sigma $ is minimum. Highlight that portfolio. We can consider that portfolio as an approximate solution to an optimization problem (find the portfolio with the no short selling constraint and achieves at least a prescribed expected return that minimizes risk *as measured by standard deviation*). For that optimal portfolio, compute its Sharpe ratio and $ \beta $.


## Section 3 - V@R and CV@R

1. Use the same dataset you collected for the previous problem. Consider the returns over one year for each of your stocks, plus S&P 500.

  (a) Construct histograms for the returns with 50 bins, and plot them, for each of your stocks.

  (b) Compute the V@R$_{1-\alpha}$ and CV@R$_{1-\alpha}$ for $\alpha = 5\%$ and $10\%$, directly from the daily returns over the year, for each stock separately. **Note:** In practice, 5% and 1% are more common, but given the limited number of data points available, we are not likely to obtain reasonable results for 1%.

2. Again, we are using the same dataset.

  (a) Given the set of daily returns for the five stocks over one year, and a weight vector $ \mathbf{w} $ for a portfolio, write code to compute the V@R and CV@R for prescribed $\alpha$.

  (b) Compute the V@R$_{1-\alpha}$ and CV@R$_{1-\alpha}$ for the MVP, MP, and S&P 500.

  (c) Use the **SAME** set of 100 random weight vectors with no short selling as you used from the MPT section of this assignment. You already have the expected returns. Now compute the CV@R$_{1-\alpha}$ for $\alpha = 10\%$ and $\alpha = 5\%$ for each of them.

  (d) Obtain a scatter plot with axes $(CV@R_{1-\alpha}, \mu)$ for your 100 weight vectors, plus the MVP, MP, S&P 500, and each of your individual stocks, for $\alpha = 10\%$. Obtain a separate scatter plot for $\alpha = 5\%$.


  (f) You now have three (estimated) "optimal" portfolios obtained under 3 different risk measures: standard deviation, and CV@R at two levels. Summarize in a table for these 3 the following: the expected return, Sharpe ratio, $\beta$, standard deviation, and both CV@R values.

## Section 4 - Comment

You obtained three scatter plots corresponding to three criteria for optimization. The "upper bound" on your scatter plots represent, essentially, "efficient frontiers" with respect to these risk measures (i.e., for a given return, the risk measure is minimized). Recall that for the standard deviation, this "upper bound" is a hyperbola, and specifically is a concave curve. Do your results appear to be concave in each of the other cases?

Here, I asked you to create 100 random portfolios. You may want to try more, if your computer can handle it. If you work with 1000, do your scatter plots and final "optimal" choices change considerably?

One more comment: in many cases, the V@R and CV@R are computed using the formulas that apply for the case of a Gaussian model. This may be useful in achieving a more systematic approach to solving the optimization problem, rather than the crude "random search" given here, but again presumes the Gaussian model is accurate.